In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rahul\\Downloads\\sign_language_conversion\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'c:\\Users\\rahul\\Downloads\\sign_language_conversion'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    test_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    testing_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from sign_language_conversion.constants import *
from sign_language_conversion.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        testing = self.config.testing
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "dataset/Train")
        testing_data = os.path.join(self.config.data_ingestion.unzip_dir, "dataset/Test")
        create_directories([
            Path(training.root_dir),
            Path(testing.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            test_dir=Path(testing.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            testing_data=Path(testing_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import tensorflow as tf

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            preprocessing_function=tf.keras.applications.vgg16.preprocess_input
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.testing_data,
            subset="validation",
            shuffle=False,
            target_size=(224, 224),
            color_mode="rgb",
            batch_size=32,
            class_mode="categorical"
        )

        train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            brightness_range=(0.1, 0.4),
            zoom_range=0.4,
            channel_shift_range=0.2,
            fill_mode='nearest',
            horizontal_flip=True,
            vertical_flip=True,
            rescale = 1./255
            )

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            shuffle=True,
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            color_mode="rgb",
            class_mode="categorical"
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        
        self.model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-05-19 09:53:30,400: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-19 09:53:30,407: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-19 09:53:30,409: INFO: common: created directory at: artifacts]
[2024-05-19 09:53:30,411: INFO: common: created directory at: artifacts\training]
[2024-05-19 09:53:30,414: INFO: common: created directory at: artifacts\testing]
[2024-05-19 09:53:31,086: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 0 images belonging to 28 classes.
Found 28000 images belonging to 28 classes.


KeyError: 'Exception encountered when calling Functional.call().\n\n\x1b[1m2643547483280\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)\n  • training=True\n  • mask=None'